In [47]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [48]:
from classes.grid import Grid

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = example

# Part 1

In [49]:
class Warehouse(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        self.robot_pos = self.get_tiles('@').pop()

    def apply_robot_instruction(self, dir):
        '''
        Moves the robot (and potentially some boxes) once in the direction dir
        '''
        match dir:
            case '^':
                dir_format = 'up'
            case '<':
                dir_format = 'left'
            case '>':
                dir_format = 'right'
            case 'v':
                dir_format = 'down'

        # Find how far away the next wall is in this direction, and how long the immediate chain of boxes is
        wall_steps_away = 1
        box_chain_on = True
        boxes_directly_in_front = 0

        while (True):
            # What tile is [iteration x] steps away in this direction?
            look_ahead = self.get_relative(self.robot_pos[0],self.robot_pos[1],dir_format,step=wall_steps_away)

            # Track how many boxes are directly in front
            if look_ahead[2] == 'O' and box_chain_on:
                boxes_directly_in_front += 1
            else:
                box_chain_on = False
            
            # Stop once we hit a wall
            if look_ahead[2] == '#':
                break
            wall_steps_away += 1

        # Look ahead in this direction up to the wall, and push all concurrent tiles
        # before a . up one square, and replace the existing tiles with the one that is one place before
        
        # Skip if we are squished up against a wall
        if boxes_directly_in_front == wall_steps_away-1:
            return


        # Loop through how long the current chain of boxes is (go in reverse to preserve grid info)
        for tile_shift in range(boxes_directly_in_front, -1, -1):

            # Move each box up one position in the direction that we're moving
            match dir:
                case '^':
                    self.grid[self.robot_pos[0]-tile_shift-1][self.robot_pos[1]] = self.grid[self.robot_pos[0]-tile_shift][self.robot_pos[1]]
                case 'v':
                    self.grid[self.robot_pos[0]+tile_shift+1][self.robot_pos[1]] = self.grid[self.robot_pos[0]+tile_shift][self.robot_pos[1]]
                case '<':
                    self.grid[self.robot_pos[0]][self.robot_pos[1]-tile_shift-1] = self.grid[self.robot_pos[0]][self.robot_pos[1]-tile_shift]
                case '>':
                    self.grid[self.robot_pos[0]][self.robot_pos[1]+tile_shift+1] = self.grid[self.robot_pos[0]][self.robot_pos[1]+tile_shift]
            
            # If we've reached the end of the chain of consecutitve boxes, then we have reached the robot,
            # so we update its position
            if tile_shift == 0:
                self.grid[self.robot_pos[0]][self.robot_pos[1]] = '.'
                self.robot_pos = self.get_tiles('@').pop()
                break
    
    def gps_sum(self):
        '''
        Returns the sum of all boxes' GPS coordinates of the current grid
        '''
        boxes = self.get_tiles('O')

        total_gps_sum = 0
        for box in boxes:
            total_gps_sum += box[0]*100 + box[1]
            
        return total_gps_sum


In [50]:
warehouse_info = input.split('\n\n')

warehouse_map_input = warehouse_info[0].split('\n')
robot_directions = ''.join(warehouse_info[1].split('\n'))

warehouse_map = Warehouse(warehouse_map_input)
#warehouse_map.print_grid()

for dir in robot_directions:
    #print(f'moving {dir}')
    warehouse_map.apply_robot_instruction(dir)
#warehouse_map.print_grid()
warehouse_map.gps_sum()

908

# Part 2

In [65]:
class WideWarehouse(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        
        # Need to re-initialise the grid in part 2 to widen everything
        new_grid = [['.']*self.num_cols*2 for _ in range(self.num_rows)]

        for row in range(self.num_rows):
            for col in range(self.num_cols):
                match self.grid[row][col]:
                    case '#':
                        new_grid[row][col*2] = '#'
                        new_grid[row][col*2+1] = '#'
                    case '.':
                        new_grid[row][col*2] = '.'
                        new_grid[row][col*2+1] = '.'
                    case 'O':
                        new_grid[row][col*2] = '['
                        new_grid[row][col*2+1] = ']'
                    case '@':
                        new_grid[row][col*2] = '@'
                        new_grid[row][col*2+1] = '.'
        
        self.grid = new_grid
        self.num_cols *= 2



In [67]:
warehouse_info = input.split('\n\n')

warehouse_map_input = warehouse_info[0].split('\n')
robot_directions = ''.join(warehouse_info[1].split('\n'))

warehouse_map = WideWarehouse(warehouse_map_input)
#warehouse_map.print_grid()